### Questo file ha il compito di generare una tabella contenente tutti gli attributi utili dei tre file, vanno inoltre settatati a missing value tutti gli attributi semanticamente scorretti (eccezzione fatta per i campi notes e incident characteristic1), infine vanno tolti i record sopra una percentuale di valori null.

In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
tab1= pd.read_csv("data/incidents.csv",low_memory=False).drop(columns= ["city_or_county","address","state_house_district","state_senate_district","participant_age_group1","incident_characteristics2"])
tab2= pd.read_csv("data/povertyByStateYear.csv",low_memory=False)
tab3 = pd.read_csv("data/year_state_district_house.csv",low_memory = False)


Filliamo tutti i campi null di incidents_characteristic1 e notes con stringhe "NULL"

In [53]:
tab1["notes"]=tab1["notes"].replace(to_replace=np.nan,value="NULL")
tab1["incident_characteristics1"]=tab1["incident_characteristics1"].replace(to_replace=np.nan,value="NULL")

In [54]:
tab1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239677 entries, 0 to 239676
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date                       239677 non-null  object 
 1   state                      239677 non-null  object 
 2   city_or_county             239677 non-null  object 
 3   address                    223180 non-null  object 
 4   latitude                   231754 non-null  float64
 5   longitude                  231754 non-null  float64
 6   congressional_district     227733 non-null  float64
 7   state_house_district       200905 non-null  float64
 8   state_senate_district      207342 non-null  float64
 9   participant_age1           147379 non-null  float64
 10  participant_age_group1     197558 non-null  object 
 11  participant_gender1        203315 non-null  object 
 12  min_age_participants       164879 non-null  object 
 13  avg_age_participants       16

left join dei file yearState e poverty su incidents. In questo mod si perdono i record con attributo year superiore al 2018

In [91]:
#creo la colonna year anche per la tabella incidents, così da poter fare la join
def toYear(date):
    if date == None:
        return np.nan
    return pd.to_datetime(date).year
tab3["state"]=tab3["state"].apply(lambda x: x.title())
tab1["year"]=tab1["date"].apply(toYear)
tab = pd.merge(tab1,tab2,on=["state","year"],how="left")
tab = pd.merge(tab,tab3, on=["year","state","congressional_district"],how="left").drop(columns=["year"])

puliamo i campi della tabella **incidents** dei valori semanticamente scorretti nei diversi attributi.<br>

puliamo i campi della tabella **poverty** dei valori semanticamente scorretti nei diversi attributi.

puliamo i campi della tabella **yearState** dei valori semanticamente scorretti nei diversi attributi.

In [102]:
len(tab)

239677

### Rimozione delle entry con troppi valori null

In [98]:
indexToDelete=[]
#il valore threshold è solo di esempio, si faranno prove con più valori
threshold= 10
for i=0 to len(tab):
    #controllo gli index delle entry che hanno più missing value di quelli consentiti da threshold
    if tab.loc[i:i].isnull().sum().sum() > threshold: 
        indexToDelete.add(i)

3